#### Project: SEO Helper
Idea: A simple tool to replicate SEO articles from others

In [ ]:
#STEPS
## 1. Grab XML sitemap for a given website
## 2. Parse it to get article URLs
## 3. Scrape or infer via URL the content
## 4. Recreate content using AI 
## 5. Publish on your own website

### Step 1: Get XML

In [57]:
#Pick directory to save the sitemap
import os
download_folder = 'sitemaps'  # Replace with the path to your desired folder
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

In [58]:
# We need to find the sitemap URL
import requests

#Set the header, pretend to be googlebot (optional)
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15'
}


def find_sitemap_url(robots_url, headers):
    """Attempt to find the sitemap URL in the robots.txt file."""
    try:
        response = requests.get(robots_url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError if the response was an error
        for line in response.text.splitlines():
            if line.startswith('Sitemap:'):
                return [line.split(':', 1)[1].strip()]
    except Exception as e:
        print(f"Error fetching or parsing robots.txt: {e}")
    return []

def try_common_sitemap_urls(base_url, headers):
    """Attempt to directly access common sitemap URLs."""
    common_paths = ['/sitemap.xml', '/sitemap_index.xml', '/page-sitemap.xml', '/post-sitemap.xml', '/post-sitemap2.xml','/event-sitemap.xml','/news-sitemap.xml','/sitemap_index.xml','/prodcut-sitemap.xml','/prodcuts-sitemap.xml','/partners-sitemap.xml']
    found_sitemaps = []
    for path in common_paths:
        url = base_url.rstrip('/') + path
        try:
            response = requests.head(url, headers=headers)
            if response.status_code == 200:
                found_sitemaps.append(url)
        except Exception as e:
            print(f"Error checking URL {url}: {e}")
    return found_sitemaps

In [59]:
try_common_sitemap_urls('https://freshprinceofstandarderror.com/', headers)

['https://freshprinceofstandarderror.com/sitemap_index.xml',
 'https://freshprinceofstandarderror.com/page-sitemap.xml',
 'https://freshprinceofstandarderror.com/post-sitemap.xml',
 'https://freshprinceofstandarderror.com/sitemap_index.xml']

In [65]:
#Download the sitemap to your directory
from urllib.parse import urlparse

## Use urlparse to get a decent name for the file
### Example
print(f'Example: {urlparse('https://a-e-i-o.com/sitemap.xml').netloc+urlparse('https://a-e-i-o.com/sitemap.xml').path}')

def download_sitemap(url, headers, folder):
    """Download the sitemap content and save to the specified folder."""
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Ensure we notice bad responses
    
    # Use urlparse to get the domain and path to create a unique filename
    url_parts = urlparse(url)
    domain = url_parts.netloc.replace("www.", "")  # Remove 'www.' if present
    path = url_parts.path.lstrip('/').replace('/', '_')  # Replace slashes with underscores
    filename = f"{domain}_{path}" if path else domain  # Construct filename using domain and path
    
    file_path = os.path.join(folder, filename)
    
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded sitemap: {filename}")

#Download whatever you find.
for maps in try_common_sitemap_urls('https://freshprinceofstandarderror.com', headers):
    download_sitemap(maps, headers, download_folder)

Example: a-e-i-o.com/sitemap.xml
Downloaded sitemap: freshprinceofstandarderror.com_sitemap_index.xml
Downloaded sitemap: freshprinceofstandarderror.com_page-sitemap.xml
Downloaded sitemap: freshprinceofstandarderror.com_post-sitemap.xml
Downloaded sitemap: freshprinceofstandarderror.com_sitemap_index.xml


### Step 2: Parse XML to get all the URLs

In [67]:
from lxml import etree

def extract_urls_from_sitemap(file_path):
    """Extracts <loc> URLs from a sitemap XML file."""
    with open(file_path, 'rb') as f:
        tree = etree.parse(f)
    
    # Define the namespace map to use with XPath
    ns_map = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    
    # Find all <loc> elements in the XML file
    loc_elements = tree.xpath('//ns:url/ns:loc', namespaces=ns_map)
    
    urls = []
    for loc in loc_elements:
        # Extract the text content within CDATA
        url = loc.text
        urls.append(url)
    
    return urls


# Call the function and print the results
urls = extract_urls_from_sitemap('sitemaps/freshprinceofstandarderror.com_post-sitemap.xml')
for url in urls:
    print(url)


https://freshprinceofstandarderror.com/?page_id=8
https://freshprinceofstandarderror.com/product/analytics-set-up-for-everybody/
https://freshprinceofstandarderror.com/uncategorized/draft-setting-up-git-and-github-for-unity-projects/
https://freshprinceofstandarderror.com/ai/games-with-reinforcement-learning/
https://freshprinceofstandarderror.com/finance/fear-and-greed-index-data-in-python/
https://freshprinceofstandarderror.com/general/12-days-of-christmath/
https://freshprinceofstandarderror.com/ai/fashion-mnist-computer-vision/
https://freshprinceofstandarderror.com/general/sql-for-everybody/
https://freshprinceofstandarderror.com/ai/unsupervised-clustering-to-understand-your-users/
https://freshprinceofstandarderror.com/ai/setting-up-your-data-science-environment/
https://freshprinceofstandarderror.com/product/automating-your-python-scripts-in-the-cloud/
https://freshprinceofstandarderror.com/ai/robust-principal-component-analysis/
https://freshprinceofstandarderror.com/product/sl

### Scrape or Infer via URL the content

In [72]:
#Put competitors in the CSV file to be organized. You just need the URL of their website.
import pandas as pd
df = pd.read_csv('competition.csv',sep=',',header=0)
df.head()

,Name,URL,Description,Known Sitemap
0,FPSE,https://freshprinceofstandarderror.com/,AI Blog,NaN
1,FPSE,https://freshprinceofstandarderror.com/,AI Blog,NaN


In [88]:
dem_urls = []
#Drop duplicates incase you have any and go through all the functions
for url in df['URL'].drop_duplicates().to_list():
    for xmaps in try_common_sitemap_urls(url, headers):
        download_sitemap(xmaps, headers, download_folder)

sitemap_files = [f for f in os.listdir(download_folder) if f.endswith('.xml')]
for file in sitemap_files:
    urls = extract_urls_from_sitemap(f'{download_folder}/{file}')
    for url in urls:
        dem_urls.append(url)

Downloaded sitemap: freshprinceofstandarderror.com_sitemap_index.xml
Downloaded sitemap: freshprinceofstandarderror.com_page-sitemap.xml
Downloaded sitemap: freshprinceofstandarderror.com_post-sitemap.xml
Downloaded sitemap: freshprinceofstandarderror.com_sitemap_index.xml


In [107]:
# Cleaning the URLs by stripping whitespace
clean_urls = [url.strip() for url in dem_urls]


#Scrape or not to scrape?
from bs4 import BeautifulSoup

def scrape_title_and_text(url):
    try:
        # Fetch the content from url
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract title
        title = soup.title.text if soup.title else 'No Title Found'

        # Extract all text
        texts = soup.get_text(separator=' ', strip=True)
        
        return title, texts
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return 'Error', 'Error'

## Lets hand pick a few links, you can do as many, dont be a prick though.
articles_scraped = []
for url in clean_urls[1:3]:
    title, all_text = scrape_title_and_text(url)
    articles_scraped.append({'URL': url, 'Title': title, 'Text': all_text})

df_articles = pd.DataFrame(articles_scraped)
df_articles.head()

,URL,Title,Text
0,https://freshprinceofstandarderror.com/product...,Web analytics set-up for everybody - Product,Web analytics set-up for everybody - Product f...
1,https://freshprinceofstandarderror.com/uncateg...,Draft: Setting up git and github for unity pro...,Draft: Setting up git and github for unity pro...


### 4. Yikes, recreate using AI

In [141]:
from openai import OpenAI
from dotenv import load_dotenv #optional, just use your key
load_dotenv()
client = OpenAI(api_key=os.getenv('OAI_API_KEY'))

prooomp = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello!"}
            ]

#You can use GPT 4 if you're loaded "gpt-4-0125-preview"
def get_article(prompt,model="gpt-3.5-turbo"):
    completion = client.chat.completions.create(
    model=model,
    messages=prompt,
    temperature=0.7,
    seed=0)
    return completion.choices[0].message.content

get_article(prooomp)

'Hello! How can I assist you today?'

In [146]:
#Create a half baked prompt that puts in the article
personality = 'Goth cheerleader that is really into tech and darkness with a lot of personality'
article_prompt = [
            {"role": "system", "content": f"You are a SEO content writer and {personality}. You have this draft that you will edit and make it sound better for your cool blog. Also make it concise and to the point. Return back markdown format and remove all hyperlinks."},
            {"role": "user", "content": f"Article:{df_articles['Text'][0]}"}
            ]

article = get_article(article_prompt)

In [147]:
from IPython.display import Markdown, display
display(Markdown(article))


article

# Web Analytics Set-Up Made Easy for Everyone

Hey there! Setting up web analytics and optimization tools may seem overwhelming, especially if you're not an expert in data or product management. But fear not, it's not as daunting as it appears. It all begins with **Google Tag Manager**.

At many of my previous workplaces, we utilized a containerized tag management system like **Google Tag Manager (GTM)**. It simplifies trying out new data collection tools and onboarding UX tools swiftly without relying on developers for every new tool.

Here's a quick rundown to get started with Google Tag Manager:

1. Create a Google Tag Manager account (selling your soul to Google is optional 😉).
2. Install a reputable plugin on your website to insert GTM snippets.
3. Utilize tools like **Google Analytics**, **Hotjar**, onboarding tools, live chat support, and optimization tools.

Setting up **Google Analytics**:

1. Create a new tag in GTM.
2. Configure the tag for Google Analytics using GA4 for enhanced features.
3. Input your tracking code (create a Google Analytics account if you don't have one yet).
4. Publish the tag in GTM to make the changes live.

Setting up **Hotjar**:

1. Sign up for a free Hotjar account.
2. Follow the onboarding process to receive a code snippet.
3. Set up session targeting to capture data effectively.

By mastering these basics, you can expand your knowledge to incorporate more cutting-edge tools used by forward-thinking organizations. Remember:

- Install Google Tag Manager.
- Add new tags and configure essential tools like GA and Hotjar.
- Dive deeper into custom triggers in GTM and effectively use analytics tools.

And hey, if you're into data analysis, why not explore setting up your python environment as well? Check out more cool stuff on my blog!

Stay tech-savvy and embrace the darkness! 🖤🦇

"# Web Analytics Set-Up Made Easy for Everyone\n\nHey there! Setting up web analytics and optimization tools may seem overwhelming, especially if you're not an expert in data or product management. But fear not, it's not as daunting as it appears. It all begins with **Google Tag Manager**.\n\nAt many of my previous workplaces, we utilized a containerized tag management system like **Google Tag Manager (GTM)**. It simplifies trying out new data collection tools and onboarding UX tools swiftly without relying on developers for every new tool.\n\nHere's a quick rundown to get started with Google Tag Manager:\n\n1. Create a Google Tag Manager account (selling your soul to Google is optional 😉).\n2. Install a reputable plugin on your website to insert GTM snippets.\n3. Utilize tools like **Google Analytics**, **Hotjar**, onboarding tools, live chat support, and optimization tools.\n\nSetting up **Google Analytics**:\n\n1. Create a new tag in GTM.\n2. Configure the tag for Google Analytics u

In [161]:
from IPython.display import Image, display

# Generate a cover image using the first few words of the article

response = client.images.generate(
  model="dall-e-3",
  prompt=article[0:100],
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

display(Image(url=image_url))


## Upload thousands of articles directly to your CMS



##### Wait.. This is too powerful and dangerous. I can't show any more. You can sabotage websites, make money from clicks, take over thousands of copywriting jobs and be a big prick. Think that is far-fetched? This article is actually to raise awareness of what people are already doing and what is possible.